Домашняя работа к лекции 2 по курсу Рекомендательные системы.
---------
**Рекомендации на основе содержания**

Студента: **Помойницкого Евгения**

Группа: **DS 8**

ДЗ - Требуется построить модель рекомендаций на основе скрытых факторов (implicit) на основе dataset’а https://grouplens.org/datasets/hetrec-2011/ (Delicious Bookmarks)”

In [40]:
import pandas as pd
import numpy as np
import scipy.sparse as sparse

from implicit.als import AlternatingLeastSquares

Попробуем понять, что за данные у нас на входе, в чем их смысл.

In [195]:
bookmarks = pd.read_table('bookmarks.dat', sep='\t', encoding='latin-1')
bookmarks.head()

,id,md5,title,url,md5Principal,urlPrincipal
0,1,ab4954b633ddaf5b5bba6e9b71aa6b70,IFLA - The official website of the Internation...,http://www.ifla.org/,7f431306c428457bc4e12b15634484f,www.ifla.org
1,2,2221e9cd106d269dd34682666f576fa3,gcdp-e.pdf (application/pdf Object),http://archive.ifla.org/VII/s14/nd1/gcdp-e.pdf,1ef8cfcfe968101fa9b4e301847503d4,archive.ifla.org
2,7,c97c571dadaddbbb493126a0d4d01ba3,EdSelect,http://www.edselect.com/,792fd7eb20143386d0c4eb193c6124d,www.edselect.com
3,8,25bfe8dca0ef263ec9c341b9f16c38b5,Cool Canada (Collections Canada),http://www.collectionscanada.gc.ca/cool/index-...,6fce4f6391516f0732531d9cfacda5b7,www.collectionscanada.gc.ca
4,9,c97284629e17b8e2861afaacd59918bc,Kidsreads.com,http://www.kidsreads.com/,5854ce8404857a45373eea01a3d98000,www.kidsreads.com


Адрес страницы - целевая фича, значение которой мы в итоге должны будем рекомендовать пользвателю. 

В таблице есть готовая колонка 'id', возможно это уже готовый идентификатор, который можно будет использовать в матрице. Проверим. Для этого сгруппируем по 'id' с подсчетом количества значений. 

In [15]:
bookmarks.groupby('id').count().sort_values('urlPrincipal', ascending=False)

,md5,title,url,md5Principal,urlPrincipal
id,,,,,
1,1,1,1,1,1
70565,1,1,1,1,1
70543,1,1,1,1,1
70544,1,1,1,1,1
70545,1,1,1,1,1
...,...,...,...,...,...
33882,1,1,1,1,1
33883,1,1,1,1,1
33884,1,1,1,1,1


In [93]:
bookmarks.groupby('urlPrincipal').count().sort_values('id', ascending=False)

,id,md5,title,url,md5Principal
urlPrincipal,,,,,
www.youtube.com,1048,1048,1048,1048,1048
www.nytimes.com,639,639,639,639,639
en.wikipedia.org,585,585,585,585,585
www.guardian.co.uk,484,484,484,484,484
mashable.com,449,449,449,449,449
...,...,...,...,...,...
sjl.funnyordie.com,1,1,1,1,1
sjctechtimes.wordpress.com,1,1,1,1,1
sixpillarstopersia.wordpress.com,1,1,1,1,1


Видно, что изрядное количество главных страниц сайтов содержат большое количество страниц следующих уровней, что не облегчает задачу.

Посмотрим содержимое других таблиц.

In [94]:
tags = pd.read_table('tags.dat', sep='\t', encoding='latin-1')
tags.head()

,id,value
0,1,collection_development
1,2,library
2,3,collection
3,4,development
4,5,lesson_plan


In [4]:
bookmark_tags = pd.read_table('bookmark_tags.dat', sep='\t', encoding='latin-1')
bookmark_tags.head()

,bookmarkID,tagID,tagWeight
0,1,2,276
1,1,206,48
2,1,208,165
3,1,272,55
4,1,459,67


In [5]:
user_taggedbookmarks = pd.read_table('user_taggedbookmarks.dat', encoding='latin-1')
user_taggedbookmarks.head()

,userID,bookmarkID,tagID,day,month,year,hour,minute,second
0,8,1,1,8,11,2010,23,29,22
1,8,2,1,8,11,2010,23,25,59
2,8,7,1,8,11,2010,18,55,1
3,8,7,6,8,11,2010,18,55,1
4,8,7,7,8,11,2010,18,55,1


In [17]:
user_contacts = pd.read_table('user_contacts.dat', encoding='latin-1')
user_contacts.head()

,userID,contactID,date_day,date_month,date_year,date_hour,date_minute,date_second
0,8,28371,4,10,2010,2,14,19
1,8,40306,3,10,2010,12,33,9
2,8,85279,2,10,2010,3,10,48
3,8,91764,4,10,2010,22,43,47
4,8,97723,12,10,2010,3,28,33


Получается, что у нас есть пользователи, которые отмечают в закладки какие-то ссылки и присваивают этим ссылкам тэги. Задача рекомендационной системы, в первом приближении, составить для какого-либо пользователя перечень ссылок, которые мы можем ему рекомендовать.

Для метода implicit ALS нужна матрица, содержащая измерение, связанное с пользователем и измерение, свзязанное со ссылкой. Примем за основу утверждение, что факт проставления тега подтверждает интерес пользователя к тому или иному сайту. В качестве значений в спарс-матрице возьмем количество тэгов.  

Для решения задачи в первом приближении нам нет необходимости использовать все данные. Постороим модель на основе таблицы  user_taggedbookmarks. В качестве значения возьмем количество тегов, выставленных тем или иным пользователем той или иной закладке.

In [95]:
df = user_taggedbookmarks.groupby(by=['userID','bookmarkID'], as_index=False, sort=False).count()
df

,userID,bookmarkID,tagID,day,month,year,hour,minute,second
0,8,1,1,1,1,1,1,1,1
1,8,2,1,1,1,1,1,1,1
2,8,7,3,3,3,3,3,3,3
3,8,8,3,3,3,3,3,3,3
4,8,9,2,2,2,2,2,2,2
...,...,...,...,...,...,...,...,...,...
104794,108035,30987,4,4,4,4,4,4,4
104795,108035,30989,1,1,1,1,1,1,1
104796,108035,30990,2,2,2,2,2,2,2
104797,108035,30993,2,2,2,2,2,2,2


In [96]:
df.columns

Index(['userID', 'bookmarkID', 'tagID', 'day', 'month', 'year', 'hour',
       'minute', 'second'],
      dtype='object')

In [97]:
df = df[['userID', 'bookmarkID', 'tagID']]
df

,userID,bookmarkID,tagID
0,8,1,1
1,8,2,1
2,8,7,3
3,8,8,3
4,8,9,2
...,...,...,...
104794,108035,30987,4
104795,108035,30989,1
104796,108035,30990,2
104797,108035,30993,2


In [98]:
df.isna().sum()

userID        0
bookmarkID    0
tagID         0
dtype: int64

In [177]:
users_ = list(np.sort(df.userID.unique()))
bookmarks_ = list(np.sort(df.bookmarkID.unique()))
tags_ = list(df.tagID)

In [178]:
print(len(users_), '\n', len(bookmarks_))

1867 
 69223


In [179]:
rows = np.array(df.userID)
cols = np.array(df.bookmarkID)

In [180]:
rows[:10]

array([8, 8, 8, 8, 8, 8, 8, 8, 8, 8], dtype=int64)

In [181]:
cols[:10]

array([ 1,  2,  7,  8,  9, 10, 11, 14, 15, 16], dtype=int64)

In [182]:
print(len(rows), '\n', len(cols), '\n', len(tags))

104799 
 104799 
 104799


In [183]:
df_sparse = sparse.csr_matrix( (tags, (cols, rows)))
df_sparse.shape

(107254, 108036)

In [184]:
# df_sparse = sparse.csr_matrix( (tags, (cols, rows)), shape=(len(bookmarks), len(users)))
"""Алексей, в чем-то я ошибся, не пойму почему не идет создание матрицы таким способом с уменьшением размерности. 
Запускал еще раз Ваш ноутбук с лекции, все работает.
В примере из документации, аналогичный способ проиллюстрирован, тоже работает"""

In [186]:
model = AlternatingLeastSquares(factors=50)
model.fit(df_sparse)

Посмотрим, что получилось в результате обучения

In [187]:
model.item_factors

array([[ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00, ...,
         0.0000000e+00,  0.0000000e+00,  0.0000000e+00],
       [ 1.1301860e-02,  1.1619688e-03, -2.6321951e-03, ...,
         7.2293211e-04, -1.1716173e-03, -5.4241749e-03],
       [ 1.1301858e-02,  1.1619689e-03, -2.6321951e-03, ...,
         7.2293205e-04, -1.1716172e-03, -5.4241735e-03],
       ...,
       [ 1.4946231e-04,  6.3248459e-05,  7.9766694e-05, ...,
        -1.9540316e-06,  1.3571480e-04, -8.1286504e-05],
       [ 1.4946231e-04,  6.3248444e-05,  7.9766694e-05, ...,
        -1.9540294e-06,  1.3571481e-04, -8.1286518e-05],
       [ 1.4946230e-04,  6.3248444e-05,  7.9766680e-05, ...,
        -1.9540335e-06,  1.3571480e-04, -8.1286504e-05]], dtype=float32)

Метод rank_items() выдает ранжированный список от данного на вход списка

In [188]:
userid = 8
bookmarkid = [ 1,  2,  7,  8,  9, 10, 11, 14, 15, 16]
model.rank_items(userid=userid, user_items=df_sparse, selected_items=bookmarkid)

[(14, 0.05666044),
 (7, 0.045984812),
 (8, 0.04330638),
 (9, 0.041138843),
 (11, 0.037935086),
 (15, 0.03644275),
 (16, 0.031013027),
 (10, 0.025563657),
 (1, 0.021208446),
 (2, 0.021208446)]

Метод recommend() возвращает наилучшие рекомендации для заданного пользователя

In [189]:
user_items = df_sparse.T.tocsr()
recommendations = model.recommend(userid, user_items)

In [190]:
recommendations

[(4638, 0.2658788),
 (14121, 0.21419597),
 (29077, 0.21280463),
 (29093, 0.21153024),
 (29103, 0.19591323),
 (29105, 0.19351757),
 (5952, 0.19311675),
 (29110, 0.1919235),
 (39473, 0.19121373),
 (29104, 0.18713593)]

In [192]:
rec_np = np.array(recommendations)
rec_np = rec_np[:,0].astype(int)
rec_np

array([ 4638, 14121, 29077, 29093, 29103, 29105,  5952, 29110, 39473,
       29104])

In [198]:
bookmarks[bookmarks['id'].isin(rec_np)].url

3270                           http://www.voki.com/
4212                      http://www.watchknow.org/
9887                    http://www.teachertube.com/
19839                   http://www.sldirectory.com/
19854    http://www.youtube.com/watch?v=-dnL00TdmLY
19860                   http://www.search-cube.com/
19861                     http://blachan.com/shahi/
19862                     http://www.visuwords.com/
19866        http://udleditions.cast.org/index.html
26777      http://copyrightfriendly.wikispaces.com/
Name: url, dtype: object

Таким образом, получили список рекомендованных страниц для пользователя с ид==8